In [1]:
import sympy as sp
from IPython.display import display, Math
from generatePoly import generatePoly

In [2]:
s = sp.symbols('s') # Creamos las variables que usaremos en nuestras funciones
e = sp.symbols('e', real=True, positive=True) # Si usamos constantes también las creamos
H = sp.symbols('H', cls=sp.Function) # Creamos ls funciones que usaremos.

In [3]:
def factorizeLaplace (polynomic, variable, constants = 0, allResults = False):
    # Validaciones
    if (type(polynomic) != sp.polys.polytools.Poly ):
        print("Polynomic have to be a sympy polynomic, reference: https://docs.sympy.org/latest/modules/polys/reference.html")
    if (type(variable) != sp.core.symbol.Symbol):
        print("Variable have to ve a sympy symbol, reference: https://docs.sympy.org/latest/modules/core.html?highlight=symbol#module-sympy.core.symbol")
    if (constants != 0):
        try:
            some_object_iterator = iter(constants)
        except TypeError as te:
            print(constants, 'have to be an array')
            return "error"
        for constant in constants:
            if (type(constant) != sp.core.symbol.Symbol and constant != 0):
                print("each constant of constants have to ve a sympy symbol, reference: https://docs.sympy.org/latest/modules/core.html?highlight=symbol#module-sympy.core.symbol")
    # Acá empieza
    # El grado del polinomio debe ser par, verifico eso y calculo el n que será el grado de H(s):
        
    if ( (polynomic.degree() % 2) != 0):
        print("degree of polynomic have to be par")
    
    n = int(polynomic.degree() / 2)
    
    # Genero un polinomio generico de grado n y los simbolos de sus constantes para H(s)
    poly, symbols = generatePoly(n)
    # Reemplazo H(s) por H(-s) en el polinomio
    npoly = sp.poly(poly.subs({variable:-variable}), variable)
    
    # Multiplico los polinomios. 
    polySquared = sp.poly(poly * npoly, variable)
    
    # Hago el array de coeficientes
    def coefArray (poly, dem, variable):
        result = []
        n = sp.degree(dem)
        for i in range(n+1):
            polyCoeff = poly.coeff_monomial(variable**i)
            demCoeff = dem.coeff_monomial(variable**i)
            equation = polyCoeff - demCoeff
            if( equation  != 0):
                result.append(equation)
        return result
    
    coefarray = coefArray(polySquared,polynomic, variable)
    
    # como nonlinsolve es muy lento, vamos a encargarnos de calcular el primer y último termino a mano.
    a_0 = sp.solve(coefarray[0], symbols[0])[0]
    a_n = sp.solve(coefarray[-1], symbols[-1])[0]
    
    for i,coef in enumerate(coefarray):
        coefarray[i] = coef.subs({symbols[0]:a_0, symbols[-1]:a_n})
    
    for coef in coefarray:
        display(Math(sp.latex(coef)))
    
    #Elimino de los simbolos al primero y último:
    
    symbols.pop(0)
    symbols.pop(-1)
    
    for constant in constants:
        symbols.append(constant)
        
    
    
    # Encuentro los candidatos a ser solución
    soluciones = sp.solve(coefarray, symbols, domain=sp.S.Reals)
    
    # Busco la solución donde todos los paarametros sean positivos
    def hasProperty(sol):
        return all((s.is_real and sp.GreaterThan(s, 0) for s in sol))
    
    # Si quiere todas las soluciones se las entrego
    if (allResults):
        return soluciones
    
    # Obtengo esa solución con todos los parametros positivos
    finalResult = list(filter(hasProperty, soluciones))

    display(finalResult)
    
    return finalResult
    

In [ ]:
EJ10 = sp.poly(-e**2*16*s**6-24*e**2*s**2-4*e**2+1,s)

factorizeLaplace(EJ10, s, [e], False)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>